In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from cs231n.data_utils import load_CIFAR10
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]

    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image

    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()

print 'Train data shape: ', X_train.shape
print 'Train labels shape: ', y_train.shape
print 'Validation data shape: ', X_val.shape
print 'Validation labels shape: ', y_val.shape
print 'Test data shape: ', X_test.shape
print 'Test labels shape: ', y_test.shape
print 'dev data shape: ', X_dev.shape
print 'dev labels shape: ', y_dev.shape

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


<h1><center> Vectorized Softmax </center></h1>

In [3]:
## Randomly initialising a weight matrix of shape (3073,10).
W = np.random.randn(3073, 10) * 0.0001
print 'Weights Matrix Shape : ', W.shape

Weights Matrix Shape :  (3073, 10)


In [4]:
## Computing the raw scores or performing the linear mathematical operation Wx + b.
scores = X_dev.dot(W)
print 'Raw Scores Matrix Shape : ', scores.shape

Raw Scores Matrix Shape :  (500, 10)


In [5]:
## Normalise the raw scores to avoid exponential score blow-up.
## To do so, subtract the maximum score from each score value for each image.
expScores = np.exp(scores - np.max(scores, axis = 1, keepdims = True))

## Compute the probabilities (or softmax scores) of each class.
softmaxScores = expScores/np.sum(expScores, axis = 1, keepdims = True)

In [6]:
## Creating a 1-D matrix containing the softmax score of the correct class.
corrSoftScore = np.choose(y_dev, softmaxScores.T)

In [7]:
## Computing the cross-entropy loss.
loss = -np.sum(np.log(corrSoftScore), axis = 0, keepdims = True)

In [8]:
dW = np.zeros_like(W)

## Initialising dO to softmaxScores.
dO = softmaxScores

## Computing dL/dO.
dO[np.arange(500), y_dev] -= 1

## Computing dL/dW with the help of chain rule.
dW = X_dev.T.dot(dO)

## Dividing by the number of training instances.
dW /= 500

dW

array([[ -1.83780219e+00,  -1.28133835e+00,  -2.61461858e-01, ...,
         -8.88552375e-01,  -6.12949262e-01,  -3.18552960e+00],
       [ -3.11166620e+00,  -3.43785810e-01,  -2.40751759e-01, ...,
         -6.27670871e-01,  -2.03378876e+00,  -3.12600963e+00],
       [ -5.15624672e+00,   3.84697937e-01,   8.70261059e-01, ...,
         -6.37840270e-01,  -4.11880429e+00,  -3.56071193e+00],
       ..., 
       [ -1.99038785e+00,  -4.05733061e-01,  -7.40336875e-01, ...,
         -1.42257267e+00,   3.08154559e+00,  -2.30298340e+00],
       [ -3.51350900e+00,  -1.26974336e-01,   1.97448842e-01, ...,
         -3.65957137e-01,   8.96716512e-01,  -2.75771577e+00],
       [ -2.59267190e-03,  -2.24856703e-02,   1.78566229e-02, ...,
          1.21162388e-02,  -1.71504155e-02,   7.02120972e-03]])

<h1><center> Naive Softmax </center></h1>

In [9]:
## Randomly initialising a weight matrix of shape (3073,10).
W = np.random.randn(3073, 10) * 0.0001
print 'Weights Matrix Shape : ', W.shape

Weights Matrix Shape :  (3073, 10)


In [10]:
## Initialising the gradients matrix.
dW = np.zeros_like(W)

## Initialising the gradients matrix of the logits i.e. Wx+b.
dO = np.zeros_like(scores)

## Initialising loss to zero.
loss = 0.0

## Computing number of training instances.
numTrain = X_dev.shape[0]

## Computing the number of classes.
numClasses = W.shape[1]

# for i in range(0, numTrain):
for i in range(0, 5):
     
    ## Computing the raw scores for each image.
    imgScoreMat = X_dev[i].dot(W)
    
    ## Finding the maximum class score in above array.
    maxClsScore = np.max(imgScoreMat)
    
    ## Normalise the raw scores to avoid exponential score blow-up.
    ## To do so, subtract the maximum score from each score value for each image.
    normScoreMat = imgScoreMat - maxClsScore
    
    ## Exponentiate the normalised class scores.
    expScoreMat = np.exp(normScoreMat)
    
    ## Computing the sum of all the exponentiated scores.
    expScoresSum = np.sum(expScoreMat, axis = 0, keepdims = True)
        
    ## Compute the probabilities (or softmax scores) of each class.
    imgSoftmaxScores = expScoreMat/expScoresSum
 
    ## Finding the softmax score for the correct class.
    corrSoftScore = imgSoftmaxScores[y_dev[i]]
    
    ## Computing the loss for the particular image.
    loss = loss + -np.log(corrSoftScore/np.sum(imgSoftmaxScores))
    
    ## Updating the gradients wrt the logits.
    for j in range(0, numClasses):

        if (j == y_dev[i]):

            dO = imgSoftmaxScores[j] - 1

        else:

            dO = imgSoftmaxScores[j]
        
        ## Updating the gradients wrt the weights.
        dW[:,j] += dO * X_dev[i]
    
          
print loss

12.0780652802


<h1><center> Tuning HyperParamters </center></h1>